# QSVT on Hamiltonian matrices

*   The `qm9_huckel` DataFrame is now prepared for subsequent quantum chemistry analyses, such as calculating eigenvalues, or for use in machine learning models that require molecular electronic structure information.
*   Further investigation could involve analyzing the distribution of matrix sizes in the 'H_pi' column to understand the complexity of π-systems across the dataset, and explicitly documenting the convention of empty lists for non-pi systems.

In [ ]:
# install prereqs
!pip install qiskit
!pip install rdkit
!pip install pennylane pyqsp
!pip install pyqsp

In [6]:
# import tools
import ast
import math
import pandas as pd
import numpy as np
import pennylane as qml
from rdkit import Chem
from pyqsp.angle_sequence import QuantumSignalProcessingPhases

#### Import dataset

In [4]:
# Load the QM9 Hückel Hamiltonian data
qm9_huckel = pd.read_csv('../qm9_hamiltonians.csv')

qm9_huckel.head()

,smiles,H_pi,pi_atoms
0,C,[],[]
1,N,[],[]
2,O,[],[]
3,C#C,"[[-0.0, -1.0], [-1.0, -0.0]]","[0, 1]"
4,C#N,"[[-0.0, -1.0], [-1.0, -0.5]]","[0, 1]"


#### Parse qm9 matrix

Normalizes each π-Hamiltonian and embeds it into a 2n×2n unitary via repeated qml.BlockEncode applications (Step 1).

In [12]:
def parse_pi_matrix(raw_matrix):
    text = raw_matrix.strip()
    if text == "[]":
        return None
    formatted = text.replace("]  [", "],[").replace("] [", "],[")
    return np.array(ast.literal_eval(formatted), dtype=float)

def normalize_hamiltonian(H):
    sigma_max = np.linalg.norm(H, ord=2)
    return H / sigma_max

def pad_to_power_of_two(H):
    n = H.shape[0]
    target = 1 << math.ceil(math.log2(n))
    padded = np.zeros((target, target), dtype=float)
    padded[:n, :n] = H
    return padded

#### Phase angles and projector 

Keeps the register initialized in |0⟩^{⊗n} implicitly (Step 2).

In [13]:
def chebyshev_phases(degree):
    coeffs = np.zeros(degree + 1)
    coeffs[degree] = 1.0
    phases_qsp, _, _ = QuantumSignalProcessingPhases(
        coeffs,
        method="sym_qsp",
        chebyshev_basis=True,
        signal_operator="Wx"
    )
    phases_qsvt = qml.transform_angles(phases_qsp, "QSP", "QSVT")
    parity = "even" if degree % 2 == 0 else "odd"
    return np.array(phases_qsvt), parity

def apply_P(phi, wire):
    qml.RZ(2.0 * phi, wire)

def apply_P_tilde(phi, wire):
    qml.Hadamard(wire)
    qml.RZ(2.0 * phi, wire)
    qml.Hadamard(wire)

#### QSVT Circuit Sequence

Applies the even/odd projector patterns explicitly through apply_P and apply_P_tilde sandwiched around U or U† to match the formulas in your slide (Steps 3–4).

In [16]:
def qsvt_state(matrix, phases, parity):
    register_qubits = int(math.ceil(np.log2(2 * matrix.shape[0])))
    signal_wire = register_qubits
    dev = qml.device("default.qubit", wires=register_qubits + 1)

    @qml.qnode(dev)
    def circuit():
        if parity == "even":
            apply_P_tilde(phases[0], signal_wire)
            for idx in range(1, len(phases) - 1, 2):
                qml.BlockEncode(matrix, wires=range(register_qubits))
                apply_P(phases[idx], signal_wire)
                qml.adjoint(qml.BlockEncode)(matrix, wires=range(register_qubits))
                apply_P_tilde(phases[idx + 1], signal_wire)
            qml.BlockEncode(matrix, wires=range(register_qubits))
        else:
            for idx in range(0, len(phases) - 1, 2):
                apply_P(phases[idx], signal_wire)
                qml.adjoint(qml.BlockEncode)(matrix, wires=range(register_qubits))
                apply_P_tilde(phases[idx + 1], signal_wire)
                qml.BlockEncode(matrix, wires=range(register_qubits))
        return qml.state()

    return circuit()

#### Apply QSVT to qm9

Runs directly on rows from qm9_huckel, yielding per-molecule QSVT state norms (you can extend the measurements as needed).

In [19]:
def qsvt_for_row(row, degree):
    matrix = parse_pi_matrix(row["H_pi"])
    if matrix is None:
        return None
    normalized = normalize_hamiltonian(matrix)
    padded = pad_to_power_of_two(normalized)
    phases, parity = chebyshev_phases(degree)
    state = qsvt_state(padded, phases, parity)
    return {
        "smiles": row["smiles"],
        "pi_atoms": len(ast.literal_eval(row["pi_atoms"])),
        "dimension": padded.shape[0],
        "state_norm": np.linalg.norm(state)
    }

degree = 2
pi_rows = qm9_huckel[qm9_huckel["H_pi"] != "[]"]
records = []
for _, row in pi_rows.head(5).iterrows():
    result = qsvt_for_row(row, degree)
    if result is not None:
        records.append(result)

pd.DataFrame(records)

[sym_qsp] Iterative optimization to err 1.000e-12 or max_iter 100.
iter: 001 --- err: 1.585e-01
iter: 002 --- err: 3.823e-02
iter: 003 --- err: 9.479e-03
iter: 004 --- err: 2.365e-03
iter: 005 --- err: 5.910e-04
iter: 006 --- err: 1.477e-04
iter: 007 --- err: 3.693e-05
iter: 008 --- err: 9.233e-06
iter: 009 --- err: 2.308e-06
iter: 010 --- err: 5.770e-07
iter: 011 --- err: 1.443e-07
iter: 012 --- err: 3.606e-08
iter: 013 --- err: 9.016e-09
iter: 014 --- err: 2.254e-09
iter: 015 --- err: 5.635e-10
iter: 016 --- err: 1.409e-10
iter: 017 --- err: 3.522e-11
iter: 018 --- err: 8.805e-12
iter: 019 --- err: 2.201e-12
iter: 020 --- err: 5.504e-13
[sym_qsp] Stop criteria satisfied.
[sym_qsp] Iterative optimization to err 1.000e-12 or max_iter 100.
iter: 001 --- err: 1.585e-01
iter: 002 --- err: 3.823e-02
iter: 003 --- err: 9.479e-03
iter: 004 --- err: 2.365e-03
iter: 005 --- err: 5.910e-04
iter: 006 --- err: 1.477e-04
iter: 007 --- err: 3.693e-05
iter: 008 --- err: 9.233e-06
iter: 009 --- err: 

,smiles,pi_atoms,dimension,state_norm
0,C#C,2,2,1.0
1,C#N,2,2,1.0
2,C=O,2,2,1.0
3,CC#C,2,2,1.0
4,CC#N,2,2,1.0
